In [20]:
import numpy as np
import pandas as pd
import torch
from torch_geometric.data import Data
data=np.load('data.npz')

In [21]:
# 节点特征
node_features=data['x']
# 节点类别
node_class=data['y']
# 边特征
edge_index=data['edge_index']
edge_type=data['edge_type']

In [22]:
train_mask=data['train_mask']
test_mask=data['test_mask']

In [23]:
xtrain=node_features[train_mask]
xtest=node_features[test_mask]
ytrain=node_class[train_mask]


In [24]:
unique,count=np.unique(ytrain,return_counts=True)
data_count=dict(zip(unique,count))

In [25]:
from sklearn.preprocessing import StandardScaler

# scale data
scaler = StandardScaler()
x_train = scaler.fit_transform(xtrain)
x_test = scaler.transform(xtest)

In [50]:
from lightgbm import LGBMClassifier
lgb=LGBMClassifier(
    learning_rate=0.1,
    n_estimators=1000,
    objective='binary',
    max_depth=7,
    reg_alpha=3,
    reg_lambda=3,
    feature_fraction=0.2,
)

In [51]:
lgb.fit(x_train,ytrain)

[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 4492, number of negative: 381612
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000951 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3565
[LightGBM] [Info] Number of data points in the train set: 386104, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.011634 -> initsco

LGBMClassifier(feature_fraction=0.2, max_depth=7, n_estimators=1000,
               objective='binary', reg_alpha=3, reg_lambda=3)

In [52]:
y_pred=lgb.predict_proba(xtest)[:,1]

[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [53]:
unique,count=np.unique(y_pred,return_counts=True)
data_count=dict(zip(unique,count))

In [54]:
result=pd.DataFrame({'index':test_mask,'predict':y_pred})
result.to_csv('lgb.csv')